In [ ]:
# docker run -it --name pyspark-jupy --mount type=bind,source=C:/Users/soumy/OneDrive/Coding,target=/app/data --rm -p 8888:8888 quay.io/jupyter/pyspark-notebook
# Reopen the older container - "docker start -ai pyspark-jupyter"

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo").getOrCreate()

In [4]:
df_sample = spark.createDataFrame(
    [
        ("sue", 32),
        ("li", 3),
        ("bob", 75),
        ("heo", None),
    ],
    ["first_name", "age"],
).repartition(2)

df_sample.show()

+----------+----+
|first_name| age|
+----------+----+
|       sue|  32|
|        li|   3|
|       bob|  75|
|       heo|NULL|
+----------+----+

